In [1]:

from newspaper import Article 
from bs4 import BeautifulSoup
import requests 
from datetime import datetime
import smtplib
import os
import csv
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email.header import Header
 
from email_config import gmail_pass, user, host, port #python file that contains user log in, host, port, etc.

def send_email_w_attachment(to, subject, body, filename):
    # create message object
    message = MIMEMultipart()
 
    # add in header
    message['From'] = Header(user)
    message['To'] = Header(to)
    message['Subject'] = Header(subject)
 
    # attach message body as MIMEText
    message.attach(MIMEText(body, 'plain', 'utf-8'))

    # locate and attach desired attachments
    att_name = os.path.basename(filename)
    _f = open(filename, 'rb')
    att = MIMEApplication(_f.read(), _subtype="txt")
    _f.close()
    att.add_header('Content-Disposition', 'attachment', filename=att_name)
    message.attach(att)
 
    # setup email server
    server = smtplib.SMTP_SSL(host, port)
    server.login(user, gmail_pass)

    # send email and quit server
    server.sendmail(user, to, message.as_string())
    server.quit()


url = "http://rss.cnn.com/rss/cnn_latest.rss" #"http://www.npr.org/rss/rss.php?id=1004"#"http://rss.cnn.com/rss/cnn_topstories.rss"
#alt URLs : http://feeds.bbci.co.uk/news/world/rss, http://rss.cnn.com/rss/cnn_topstories.rss, http://www.aljazeera.com/xml/rss/all.xml, http://rss.cnn.com/rss/edition_world.rss
# http://www.npr.org/rss/rss.php?id=1004, http://www.independent.co.uk/news/world/rss, http://feeds.foxnews.com/foxnews/national
# "https://rss.app/feeds/l9hGuJhMQJTY7N57.xml"

resp = requests.get(url)
soup = BeautifulSoup(resp.content, features="xml")

items = soup.findAll('item')
links = soup.findAll('link')
titles = soup.findAll('title')
pubDate = soup.findAll('pubDate')
news_items = []

for item in items:
    news_item = {}
    news_item['title'] = item.title.text
    news_item['link'] = item.link.text
    news_item['pubDate'] = item.pubDate
    #news_item['pic'] = item.content['url']
    news_items.append(news_item)

counter = 0
now = datetime.today()

header = "Top 10 News Items as of " + str(now)
blank = '\n'
title = '\n' + "Article's Title: "
link = '\n' + "Article's Link:"
summary = '\n' + "Article's Summary: "
stars = '\n' + "*****************************************************"
bars = "/////"
body = ""

#Future enhancement for replying with what news feed the user wants
disclaimer = "Reply to this email with these options for custom summaries:" + blank
bbc = "1 - BBC" + blank
cnn = "2 - CNN Top Stories" + blank
aljazeera = "3 - Al-Jazeera" + blank
cnnWorld = "4 - CNN World" + blank
npr = "5 - NPR" + blank
independent = "6 - UK Independent" + blank
fox = "7 - Fox" + blank
custom = "8 - My Custom URL" + blank
####

filename = 'EmailDatabase.csv'
subject = header
attachment = "NewsOutput.txt"

open('NewsOutput.txt', 'w').close()
f = open("NewsOutput.txt", "a")

#create header

#print (bars)
print (bars, file=f)
body += bars

#print (header)
print(header, file=f)
body += header

#print (bars)
print (bars, file=f)
body += bars + blank + blank

body += disclaimer + bbc + cnn + aljazeera + cnnWorld + npr + independent + fox + custom + stars + blank

for each in news_items:
    #print (title + blank + each['title'])
    print (title + blank +  each['title'], file=f)
    body += title + blank + each['title'] + blank

    url = each['link']

    #print (link)
    print (link, file=f)
    body += link + blank

    #print (url)
    print (url, file=f)
    body += url + blank

    toi_article = Article(url, language="en")
    toi_article.download()
    toi_article.parse() 
    toi_article.nlp() 
    toi_article.top_image    

    #print(summary)
    #print(toi_article.summary)
    # print (summary, file=f)
    # print (toi_article.summary, file=f)
    body += summary + blank
    body += toi_article.summary + blank 

    #print (stars)
    print (stars, file=f)
    body += stars + blank

    counter += 1
    if counter == 10:
        break

f.close()


######Send Email to those in csv

with open(filename, 'r') as csvfile:
    datareader = csv.reader(csvfile)
    next (datareader)

    for row in datareader:
        to = row[1]
        send_email_w_attachment (to, subject, body, attachment)

    print ("sent")

sent
